In [5]:
from boggle.io import download_txt_file, get_word_list, process_word_list, calculate_letter_probs_from_word_list
from boggle.path import Path
from boggle.boggle import BoggleGrid, Boggle
import numpy as np
import random
import time
import re

In [6]:
MIN_WORD_LENGTH = 3
WORDLIST = '../data/basiswoorden-gekeurd.txt'

## Data

In [7]:
download_txt_file(txt_url='https://github.com/OpenTaal/opentaal-wordlist/raw/master/elements/basiswoorden-gekeurd.txt', 
                  target_path='../data/basiswoorden-gekeurd.txt')
word_list = np.array(get_word_list(txt_path=WORDLIST))
word_list = process_word_list(word_list, min_word_length=MIN_WORD_LENGTH)
letter_probs = calculate_letter_probs_from_word_list(word_list)

## Utils

In [10]:
boggle_grid = BoggleGrid(letter_probs = letter_probs, gridsize = (4,4))

## Code

In [11]:
boggle = Boggle(boggle_grid,word_list)
all_words = boggle.solve()

print('--- BOGGLE ---\n')
print(boggle_grid)

print('\n\n--- ALL WORDS ---\n')
print([word['word'] for word in all_words])

--- BOGGLE ---

[['e' 't' 'b' 'p']
 ['v' 'l' 'u' 'o']
 ['r' 'i' 'n' 's']
 ['k' 'c' 'a' 'g']]


--- ALL WORDS ---

['tui', 'tul', 'tub', 'tel', 'bos', 'bon', 'bop', 'bus', 'bun', 'bui', 'bul', 'pos', 'pon', 'pus', 'pui', 'pul', 'put', 'pub', 'vin', 'via', 'vet', 'vel', 'lus', 'lui', 'lut', 'lno', 'lic', 'lik', 'lev', 'lbo', 'uso', 'uni', 'uil', 'ons', 'out', 'ria', 'ril', 'ino', 'nag', 'nil', 'nul', 'nut', 'nou', 'nop', 'sul', 'sub', 'sup', 'son', 'sou', 'sop', 'kca', 'kin', 'kir', 'kil', 'cai', 'cas', 'air', 'aso', 'gas', 'etui', 'tuin', 'tuil', 'tule', 'bons', 'boni', 'bout', 'buso', 'buik', 'buil', 'bult', 'blut', 'blik', 'pons', 'puin', 'puik', 'vlet', 'ving', 'vilt', 'luns', 'luik', 'lino', 'lias', 'lick', 'live', 'onus', 'obus', 'opus', 'rins', 'ring', 'nick', 'snik', 'snul', 'snob', 'song', 'soul', 'king', 'kina', 'kilt', 'anus', 'gans', 'bosui', 'bonus', 'poule', 'rivet', 'snack', 'kring', 'kilte', 'civet', 'tunica', 'teling', 'bosuil', 'bonsai', 'poulet', 'lunair', 'insult', 'c